<a href="https://colab.research.google.com/github/tingwry/Advanced-Com-Prog-Lab/blob/main/Lab%3A%20Chart%20in%20Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 7: Chart in Flask**

Please save the notebook into your drive; work on the saved one; download the finished one; submit the final file to MyCourseVille. 

## **Lab 7.1 Simple Chart**




Create a route (`/show_score`) on **your colab** to display a bar chart of all quiz scores of the student whose id will be passed in the url (`/show_score?id=xxxxxxxxxx`). The score you retrieve must be get from pymongo.

## **Lab 7.2 Create your own `hist()`**




Create the route (/**show_hist**) in **your colab** to display a histogram of total score of all students. Your service must get a bin value from url (`/show_hist?bins=10'. 

In [ ]:
! gdown https://drive.google.com/uc?id=1wpYx8rvGohvy_TLEfYgaKRYpyy2Z1ZST

Downloading...
From: https://drive.google.com/uc?id=1wpYx8rvGohvy_TLEfYgaKRYpyy2Z1ZST
To: /content/score.csv
100% 22.4k/22.4k [00:00<00:00, 19.2MB/s]


# **Your program**

In [ ]:
!pip install pymongo[srv]
!pip install flask_ngrok
!pip install flask flask-cors
!pip install pyngrok
!ngrok authtoken 2F9tYl79jvby0znP22sTdLHBXBq_6nsNcit9L5XLNmprViyow
! mkdir static 
! wget https://cdnjs.cloudflare.com/ajax/libs/Chart.js/1.0.2/Chart.min.js
! mv Chart.min.js static/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
mkdir: cannot create directory ‘static’: File exists
--2022-10-08 10:24:56--  https://cdnjs.cloudflare.com/ajax/libs/Chart.js/1.0.2/Chart.min.js
Resolving cdnjs.cloudflare.com (cdnjs.cloudflare.com)... 104.17.25.14, 104.17.24.14, 2606:4700::6811:180e, ...
Connecting to cdnjs.cloudflare.com (cdnjs.cloudflare.com)|104.17.25.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/javascript]
Saving to: ‘Chart.min.js’

Chart.min.js            [ <=>                ]  50.87K  --.-KB/s    in 0.001

In [ ]:
!mkdir templates
file=open('templates/chart.html','w')
file.write('''
<meta charset="utf-8">
<title>Chart.js </title>

<!-- import plugin script -->
<script src="/static/Chart.min.js"></script>
<h1>Flask Chart.js</h1>
<!-- bar chart canvas element -->
<canvas id="chart" width="600" height="400"></canvas>

<script>
// bar chart data
   var barData = {
   labels : [{% for item in labels %}
   "{{item}}",
{% endfor %}],
   datasets : [
      {
            fillColor: "rgba(151,187,205,0.2)",
            strokeColor: "rgba(151,187,205,1)",
            pointColor: "rgba(151,187,205,1)",
            pointStrokeColor: "#fff",
            pointHighlightFill: "#fff",
            pointHighlightStroke: "rgba(151,187,205,1)",
            bezierCurve : false,
            data : [{% for item in values %}
{{item}},
{% endfor %}]
      }]
   }
    Chart.defaults.global.animationSteps = 50;
    Chart.defaults.global.tooltipYPadding = 16;
    Chart.defaults.global.tooltipCornerRadius = 0;
    Chart.defaults.global.tooltipTitleFontStyle = "normal";
    Chart.defaults.global.tooltipFillColor = "rgba(0,0,0,0.8)";
    Chart.defaults.global.animationEasing = "easeOutBounce";
    Chart.defaults.global.responsive = false;
    Chart.defaults.global.scaleLineColor = "black";
    Chart.defaults.global.scaleFontSize = 16;
// get bar chart canvas<br />
   var mychart = document.getElementById("chart").getContext("2d");
   steps = 10
   max = {{max}}
   // draw line chart
   var LineChartDemo = new Chart(mychart).Bar(barData, {
        scaleOverride: true,
        scaleSteps: steps,
        scaleStepWidth: Math.ceil(max / steps),
        scaleStartValue: 0,
        scaleShowVerticalLines: true,
        scaleShowGridLines : true,
        barShowStroke : true,
        scaleShowLabels: true,
        bezierCurve: false,
   });
</script>
''')
file.close()

mkdir: cannot create directory ‘templates’: File exists


In [ ]:
import pymongo
import pandas as pd 
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request
from flask import Markup
from flask import Flask
from flask import render_template
from flask_cors import CORS, cross_origin

app = Flask(__name__)
CORS(app, support_credentials=True)
run_with_ngrok(app)

client = pymongo.MongoClient("############################################################################")
db = client.postdb

@app.route("/show_score")
def show_score():
  id=request.args.get('id')
  
  student=db.scores.find_one({'student_id':int(id)})
  if student is  None:
    return 'Not found'

  labels = ["Q1","Q2","Q3","Q4","Q5"]
  values = [student['Q1'],student['Q2'],student['Q3'],student['Q4'],student['Q5']]

  return render_template('chart.html', values=values, labels=labels, max=10)


@app.route("/show_hist")
def show_hist():
  df=pd.read_csv('score.csv')
  df['Total']=df['Q1']+df['Q2']+df['Q3']+df['Q4']+df['Q5']

  bins=request.args.get('bins')
  
  maxs=[]
  values=[]
  size=50/int(bins)
  for i in range(1,int(bins)+1):
    maxs.append(size*i)
    values.append(0)
    
  for index, student in df.iterrows():
    for i in range(len(maxs)):
      if student['Total'] <= maxs[i]:
        values[i] += 1
        break

  return render_template('chart.html', values=values, labels=maxs, max=max(values))


if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2401-104-196-37-131.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Oct/2022 10:25:20] "GET / HTTP/1.1" 404 -
ERROR:__main__:Exception on /show_score [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-p